## Import required libraries

In [1]:
import zipfile
import os
import sys
import pandas as pd
import openai
import json  
import wget
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
load_dotenv()
# Load OpenAI access and other custom paths
sys.path.append(os.path.abspath('../../src'))
from azure_openai_conn import OpenAIembeddings


In [2]:
from tenacity import retry, wait_random_exponential, stop_after_attempt 
from openai import AzureOpenAI
import json
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)

In [3]:
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  

In [4]:
# Configure environment variables  
service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT") 
index_name = "financebench-small-metadata" 
key = os.getenv("SEARCH_SERVICE_API_KEY") 
model: str = "text-embedding-ada-002" 
credential = AzureKeyCredential(key)

In [5]:
client = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [6]:
# Configure OpenAI settings
openai.api_type = os.getenv('OPENAI_API_TYPE')
openai.api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai.api_version = os.getenv('OPENAI_API_VERSION')
openai.api_key = os.getenv('OPENAI_API_KEY')
model: str = os.getenv('OPENAI_EMBEDDINGS_MODEL_NAME') 

In [7]:
azure_search_endpoint: str = os.getenv('SEARCH_SERVICE_ENDPOINT') 
azure_search_key: str = os.getenv('SEARCH_SERVICE_API_KEY')
index_name: str = "azure-cognitive-search-vector-demo"
credential = AzureKeyCredential(azure_search_key)

In [8]:
embeddings = OpenAIembeddings()

### FinanceBench-Small (to fit Azure AISearch cuota)

In [7]:
companies = ("MICROSOFT_2023_10K.pdf","JOHNSON&JOHNSON_2022Q4_EARNINGS.pdf","Pfizer_2023Q2_10Q.pdf","BESTBUY_2017_10K.pdf","BESTBUY_2019_10K.pdf")

In [8]:
destination_folder = '../../data/financebench'
pdf_folder_path = destination_folder
documents = []
for file in companies:
    print(file)
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

MICROSOFT_2023_10K.pdf
JOHNSON&JOHNSON_2022Q4_EARNINGS.pdf
Pfizer_2023Q2_10Q.pdf
BESTBUY_2017_10K.pdf
BESTBUY_2019_10K.pdf


In [9]:
chunk_size=1024
overlap=100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap, add_start_index=True)
chunked_documents = text_splitter.split_documents(documents)

In [10]:
len(chunked_documents)

1706

In [18]:
chunked_documents[0].to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'document', 'Document'],
 'kwargs': {'page_content': 'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n \nFORM \n10-K\n \n \n☒\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \n \n \nFor the Fiscal Year Ended\n June 30, \n2023\n \n \n \nOR\n \n \n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \n \n \nFor the Transition Period From                  to\nCommission File Number \n001-37845\n \n \nMICROSOFT CORPORATION\n \n \nWASHINGTON\n \n91-1144442\n(STATE OF INCORPORATION)\n \n(I.R.S. ID)\nONE MICROSOFT WAY\n, \nREDMOND\n, \nWASHINGTON\n \n98052-6399\n(\n425\n) \n882-8080\nwww.microsoft.com/investor\n \n \n \n \n \nSecurities registered pursuant to Section 12(b) of the Act:\n \n \n \n \n \n \n \n \n \nTitle of each class\n \nTrading Symbol\n \nName of exchange on which registered\n \n \n \n \n \nCommon stock, $\

# Create data object with vectors and save it

In [18]:
def generate_embeddings(text, model=model):
        return client.embeddings.create(input = [text], model=model).data[0].embedding


In [25]:
docVector = '../../exp/exp9_dbconfiguration-azure_ai_search/docVectors.json'

if not os.path.isfile(docVector):

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    # Function to generate embeddings for title and content fields, also used for query embeddings
    def generate_embeddings(text, model=model):
        return client.embeddings.create(input = [text], model=model).data[0].embedding

    docVectors = []

    # Generate embeddings for each page and save other fields
    for item in chunked_documents:
        
        content_embeddings = generate_embeddings(item.page_content)

        # other metadata of documents
        filename = item.metadata['source'].split('/')[4]

        company_name = filename.split('_')[0].upper() 
        doc_year = filename.split('_')[1] 
        doc_quarter = doc_year[4:len(doc_year)]
        doc_year = doc_year[:4]
        doc_type = filename.split('_')[2].replace('.pdf', '') 

        
        # Create a dictionary for the current item
        docDict = {
            'metadata': item.metadata,
            'page_content': item.page_content,
            'contentVector': content_embeddings,
            'company_name': company_name,
            'doc_year': doc_year,
            'doc_quarter': doc_quarter,
            'doc_type': doc_type
        }

        # Append the dictionary to the docVectors list
        docVectors.append(docDict)

    # Output embeddings to docVectors.json file
    with open("docVectors.json", "w") as f:
        json.dump(docVectors, f)

# Re-load documents

In [9]:
# Upload some documents to the index
with open('docVectors.json', 'r') as file:  
    documents = json.load(file)  

In [10]:
# Iterate over each document and update it
for document in documents:
    document.update(document.pop('metadata'))

In [11]:
for i, document in enumerate(documents):
    document['id'] = str(i)
    document['page'] = str(i)
    document['start_index'] = str(i)


In [12]:
first_document = documents[0] 
{key: (value[:2] if isinstance(value, list) else value) for key, value in first_document.items()}


{'page_content': 'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n \nFORM \n10-K\n \n \n☒\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \n \n \nFor the Fiscal Year Ended\n June 30, \n2023\n \n \n \nOR\n \n \n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \n \n \nFor the Transition Period From                  to\nCommission File Number \n001-37845\n \n \nMICROSOFT CORPORATION\n \n \nWASHINGTON\n \n91-1144442\n(STATE OF INCORPORATION)\n \n(I.R.S. ID)\nONE MICROSOFT WAY\n, \nREDMOND\n, \nWASHINGTON\n \n98052-6399\n(\n425\n) \n882-8080\nwww.microsoft.com/investor\n \n \n \n \n \nSecurities registered pursuant to Section 12(b) of the Act:\n \n \n \n \n \n \n \n \n \nTitle of each class\n \nTrading Symbol\n \nName of exchange on which registered\n \n \n \n \n \nCommon stock, $\n0.00000625\n par value per share\n \nMSFT\n \nNASDAQ\n3.125% Notes due 2028\n \nMSFT\n \nNASDAQ\n2.6

In [13]:
len(documents)

1706

## Create your search index
Create your search index schema and vector search configuration:

In [13]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    # Warning! too many 'filterable's field?
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="doc_year", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchableField(name="doc_type", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="source", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="company_name", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="doc_quarter", type=SearchFieldDataType.String),
    SearchableField(name="page", type=SearchFieldDataType.String),
    SearchableField(name="start_index", type=SearchFieldDataType.String),
    SearchableField(name="page_content", type=SearchFieldDataType.String), # not searchable?
    # SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
    #             searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
# https://learn.microsoft.com/en-us/python/api/azure-search-documents/azure.search.documents.indexes.models.hnswvectorsearchalgorithmconfiguration?view=azure-python-preview
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        )
    ]
)



semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="source"),
        keywords_fields=[SemanticField(field_name="doc_type")],
        content_fields=[SemanticField(field_name="company_name")]
    )
)

In [15]:
# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 azure-cognitive-search-vector-demo created


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [16]:
# Upload some documents to the index
# with open('../output/docVectors.json', 'r') as file:  
#     documents = json.load(file)  

search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents") 

Uploaded 1706 documents


If you are indexing a very large number of documents, you can use the `SearchIndexingBufferedSender` which is an optimized way to automatically index the docs as it will handle the batching for you:

In [34]:
uploadwithbuffer = False

if uploadwithbuffer:
    # Upload some documents to the index  
    with open('../output/docVectors.json', 'r') as file:  
        documents = json.load(file)  
    
    # Use SearchIndexingBufferedSender to upload the documents in batches optimized for indexing  
    with SearchIndexingBufferedSender(  
        endpoint=service_endpoint,  
        index_name=index_name,  
        credential=credential,  
    ) as batch_client:  
        # Add upload actions for all documents  
        batch_client.upload_documents(documents=documents)  
    print(f"Uploaded {len(documents)} documents in total")  


## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [19]:
# Pure Vector Search
query = "Revenue of Microsoft"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["company_name", "source", "doc_type", "page_content"],
)  
  
for result in results:  
    print(f"Company_name: {result['company_name']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Source: {result['source']}")  
    print(f"Doc_type: {result['doc_type']}")  
    print(f"Page_content: {result['page_content']}\n")
    print('-'*100)  


Company_name: MICROSOFT
Score: 0.90272355
Source: ../../data/financebench/MICROSOFT_2023_10K.pdf
Doc_type: 10K
Page_content: manufacturing, marketing, and selling our other products and services; and income taxes.
Highlights from fiscal year 2023 compared with fiscal year 2022 included:
•
Microsoft Cloud revenue increased 22% to $111.6 billion.
•
Office Commercial products and cloud services revenue increased 10% driven by Office 365 Commercial growth of 13%.
•
Office Consumer products and cloud services revenue increased 2% and Microsoft 365 Consumer subscribers increased to 67.0 million.
•
LinkedIn revenue increased 10%.
•
Dynamics products and cloud services revenue increased 16% driven by Dynamics 365 growth of 24%.
•
Server products and cloud services revenue increased 19% driven by Azure and other cloud services growth of 29%.
•
Windows original equipment manufacturer licensing (“Windows OEM”) revenue decreased 25%.
•
Devices revenue decreased 24%.
•
Windows Commercial products a

## Perform an Exhaustive KNN exact nearest neighbor search

This example shows how you can exhaustively search your vector index regardless of what index you have, HNSW or ExhaustiveKNN. You can use this to calculate the ground-truth values.

In [38]:
# Pure Vector Search  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector", exhaustive=True)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["company_name", "source", "doc_type", "page_content"],
)  
  
for result in results:  
    print(f"Company_name: {result['company_name']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Source: {result['source']}")  
    print(f"Doc_type: {result['doc_type']}")  
    print(f"Page_content: {result['page_content']}\n")
    print('-'*100)  



Company_name: MICROSOFT
Score: 0.9027239
Source: ../../data/financebench/MICROSOFT_2023_10K.pdf
Doc_type: 10K
Page_content: manufacturing, marketing, and selling our other products and services; and income taxes.
Highlights from fiscal year 2023 compared with fiscal year 2022 included:
•
Microsoft Cloud revenue increased 22% to $111.6 billion.
•
Office Commercial products and cloud services revenue increased 10% driven by Office 365 Commercial growth of 13%.
•
Office Consumer products and cloud services revenue increased 2% and Microsoft 365 Consumer subscribers increased to 67.0 million.
•
LinkedIn revenue increased 10%.
•
Dynamics products and cloud services revenue increased 16% driven by Dynamics 365 growth of 24%.
•
Server products and cloud services revenue increased 19% driven by Azure and other cloud services growth of 29%.
•
Windows original equipment manufacturer licensing (“Windows OEM”) revenue decreased 25%.
•
Devices revenue decreased 24%.
•
Windows Commercial products an

## Perform a Cross-Field Vector Search

This example shows a cross-field vector search that allows you to query multiple vector fields at the same time. Note, ensure that the same embedding model was used for the vector fields you decide to query.

In [ ]:
# # Pure Vector Search  
# search_client = SearchClient(service_endpoint, index_name, credential=credential)
# vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector, titleVector")
# # Use the below query to pass in the raw vector query instead of the query vectorization
# # vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="titleVector, contentVector")
  
# results = search_client.search(  
#     search_text=None,  
#     vector_queries= [vector_query],
#     select=["company_name", "source", "doc_type", "page_content"],
# )  
  
# for result in results:  
#     print(f"Company_name: {result['company_name']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"Source: {result['source']}")  
#     print(f"Doc_type: {result['doc_type']}")  
#     print(f"Page_content: {result['page_content']}\n")
#     print('-'*100)  



## Perform a Multi-Vector Search

This example shows a cross-field vector search that allows you to query multiple vector fields at the same time by passing in multiple query vectors. Note, in this case, you can pass in query vectors from two different embedding models to the corresponding vector fields in your index.

In [ ]:
# Multi-Vector Search
# query = "tools for software development"  
  
# search_client = SearchClient(service_endpoint, index_name, credential=credential)  
# vector_query_1 = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="titleVector")
# vector_query_2 = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
  
# results = search_client.search(  
#     search_text=None,  
#     vector_queries=[vector_query_1, vector_query_2],
#     select=["title", "content", "category"],
# )  
  
# for result in results:  
#     print(f"Title: {result['title']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"Content: {result['content']}")  
#     print(f"Category: {result['category']}\n")  


## Perform a Pure Vector Search with a filter
This example shows how to apply filters on your index. Note, that you can choose whether you want to use Pre-Filtering (default) or Post-Filtering.

In [39]:
# Pure Vector Search
query = "What is the Revenue of 2019"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="company_name eq 'BESTBUY'",
    select=["company_name", "source", "doc_type", "page_content"],
)  
  
for result in results:  
    print(f"Company_name: {result['company_name']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Source: {result['source']}")  
    print(f"Doc_type: {result['doc_type']}")  
    print(f"Page_content: {result['page_content']}\n")
    print('-'*100)  


Company_name: BESTBUY
Score: 0.8698138
Source: ../../data/financebench/BESTBUY_2019_10K.pdf
Doc_type: 10K
Page_content: 2019   2018   2017 Revenue
$ 39,304   $38,662   $36,248 Revenue % increase (decrease)
1.7 % 6.7 % (0.3 )%Comparable sales growth 
(1)4.8% 5.6 % 0.2  %Gross profit
$ 9,144  $9,065  $8,650 Gross profit as % of revenue
23.3 % 23.4 % 23.9  %SG&A
$ 7,300  $7,304  $6,855 SG&A as % of revenue
18.6 % 18.9 % 18.9  %Restructuring charges
$ 47  $9  $31 Operating income
$ 1,797  $1,752  $1,764 Operating income as % of r
evenue4.6 % 4.5 % 4.9  % 
        Selected Online Revenue Data
        Total online revenue
$6,528  $5,991  $4,843 Online revenue as a % of tot
al segment revenue16.6% 15.5 % 13.4  %Comparable online sales growth 
(1)10.5% 21.8 % 20.8  %29

----------------------------------------------------------------------------------------------------
Company_name: BESTBUY
Score: 0.86943895
Source: ../../data/financebench/BESTBUY_2019_10K.pdf
Doc_type: 10K
Page_content: Table

## Perform a Hybrid Search

In [40]:
# Hybrid Search
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["company_name", "source", "doc_type", "page_content"],
    top=3
)  
  
for result in results:  
    print(f"Company_name: {result['company_name']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Source: {result['source']}")  
    print(f"Doc_type: {result['doc_type']}")  
    print(f"Page_content: {result['page_content']}\n")
    print('-'*100)  

Company_name: BESTBUY
Score: 0.03151364624500275
Source: ../../data/financebench/BESTBUY_2019_10K.pdf
Doc_type: 10K
Page_content: e in the excess profits. In fiscal 2019, we recognized $7 million of such profit-share revenue, with an equal impact to gross profit andoperating income. In fis
cal 2018, we recognized $59 million of such profit-share revenue. The fiscal 2019 profit-share revenue decrease from fiscal 2018 reflectsreductions to the premiu
ms that we pay to the third-party underwriter.In fiscal 2019, Domestic segm
ent online revenue of $6.5  billion increased 10.5% on a comparable basis, primarily due to higher conversion rates and increasedtraffic. As a percentage of
 total Domestic revenue, online revenue increased to 16.6%  versus 15.5% last year.30

----------------------------------------------------------------------------------------------------
Company_name: BESTBUY
Score: 0.028588563203811646
Source: ../../data/financebench/BESTBUY_2019_10K.pdf
Doc_type: 10K
Page_cont

## Perform a Semantic Hybrid Search

In [41]:
# Semantic Hybrid Search

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["company_name", "source", "doc_type", "page_content"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Company_name: {result['company_name']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Source: {result['source']}")  
    print(f"Doc_type: {result['doc_type']}")  
    print(f"Page_content: {result['page_content']}\n")
    print('-'*100)  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


HttpResponseError: (FeatureNotSupportedInService) Semantic search is not enabled for this service.
Parameter name: queryType
Code: FeatureNotSupportedInService
Message: Semantic search is not enabled for this service.
Parameter name: queryType
Exception Details:	(SemanticQueriesNotAvailable) Semantic search is not enabled for this service.
	Code: SemanticQueriesNotAvailable
	Message: Semantic search is not enabled for this service.

## Integrate whit Langhcin QA RAG-Chat!!!

https://github.com/microsoft/Form-Recognizer-Toolkit/blob/main/SampleCode/Python/sample_rag_langchain.ipynb 